In [ ]:
import csv
from pathlib import Path
import requests
import sqlalchemy
import pandas as pd
from sqlalchemy import create_engine, MetaData
from sqlalchemy.dialects.postgresql import insert
#from sqlalchemy.sql import table, column, select, update, insert

In [ ]:
engine = create_engine('postgresql+psycopg2://<user>:<password>@localhost:<port>/POIU_Analytics', connect_args={'options': '-csearch-path={}'.format('poiu-comparison')})

In [ ]:
conn = engine.connect()

In [ ]:
engine.dispose()

In [ ]:
conn.close()

In [ ]:
## Change to False to disable Sample
SHOW = True

# I want to test this on a single game first
TEST_GAME_ID = 20001.0
TEST_GAME_SEASON = 2023.0


In [ ]:
# helpers
def get_full_game_id(partial_id, season):
    return f"{int(season)}0{int(partial_id)}"

In [ ]:
# is numeric wasn't perfect.
def is_number(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

In [ ]:
def convert_minute_format_to_seconds(string_time_value):
    # fix this handle none values better.
    if string_time_value is None:
        return 0.0
    minutes, seconds = string_time_value.split(':')
    total_time = float(minutes)*60 + float(seconds)
    return float(total_time)

In [ ]:
def make_rows_values_correct_type(row):
    # row is a on object because of the dictreader here.
    # essentially just converts numbers to floats
    # and keeps the strings the same.
    for key in row.keys():
        value = row[key]
        if is_number(value):
            row[key] = float(value)

    return row

In [ ]:
def get_shots_file_path():
    cwd = Path.cwd()
    shots_file_path = cwd.parent / "data" / "shots" / "shots_2023_2024.csv"
    return shots_file_path

In [ ]:
print(get_shots_file_path())

In [ ]:
# the meat of the parsing.
def import_shot_game_data(shots_file):
    print("importing shots...")

    all_shot_rows = []
    with open(shots_file, newline='') as csvfile:
        reader = csv.DictReader(csvfile)  # Uses the first line as column headers

        for row in reader:
            # converts string numbers and all nubmers to float
            row_clean = make_rows_values_correct_type(row)
            # skip if not in test game id remove when serious
            if row_clean["game_id"] != TEST_GAME_ID:
                continue


            all_shot_rows.append(row_clean)
    # get the shot data from ../data/shots/
    return all_shot_rows

In [ ]:
# note we'll have to be carful with this to not ddos them
def import_shift_data(game_id, season):
    print("importing shifts...")
    # URL building
    full_game_id = get_full_game_id(game_id, season)
    full_url = f"https://api.nhle.com/stats/rest/en/shiftcharts?cayenneExp=gameId={full_game_id}"

    #print(full_url)
    response = requests.get(full_url)

    full_shift_data = response.json()
    #print(full_shift_data["data"])
    # need to get the times of shifts in numbers
    # this is needed for the money puck data.
    for index, shift in enumerate(full_shift_data["data"]):
        #print(index)
        start_time_number = convert_minute_format_to_seconds(shift["startTime"])
        end_time_number = convert_minute_format_to_seconds(shift["endTime"])
        duration_number = convert_minute_format_to_seconds(shift["duration"])

        # update the
        full_shift_data["data"][index]["startTimeNumber"] = start_time_number
        full_shift_data["data"][index]["endTimeNumber"] = end_time_number
        full_shift_data["data"][index]["durationNumber"] = duration_number
        
    return full_shift_data["data"]

In [ ]:
def assign_players_on_ice_to_shots(shot_data, shift_data):
    print("assigning player shifts to shots...")
    for index, shot in enumerate(shot_data):
        players_for, players_against = get_players_on_ice_for_shot(shot, shift_data)
        
        shot_data[index]["playersOnIceFor"] = players_for
        shot_data[index]["playersOnIceAgainst"] = players_against

    return shot_data


In [ ]:
# there's probably a better way of doing this with sets
# but if we need to process it once and then have the data then we should be good.
def get_players_on_ice_for_shot(shot, shift_data):
    players_for = []
    players_against = []
    period = shot["period"]
    shot_time = shot["time"]
    for shift in shift_data:
        if not (shift["startTimeNumber"] < shot_time
            and shift["endTimeNumber"] > shot_time
            and shift["period"] == period):
            continue

        if shot["teamCode"] == shift["teamAbbrev"]:
            players_for.append(shift)
        else:
            players_against.append(shift)

    #players_for.sort()
    #players_against.sort()

    return players_for, players_against

In [ ]:
class shot_db_sql():

    def create_connection(self, path):
        connection = None
        try:
            #connection = psycopg2.connect(database= "postgres",
            #                                user="postgres",
            #                                host="127.0.0.1",
            #                                port="5050")
            #connection.text_factory = str
            connection = engine.connect()
        except Error as e:
            print("Error occurred: " + str(e))
    
        return connection

    def execute_query(self, connection, query):
        cursor = connection.cursor()
        try:
            if query == "":
                return "Query Blank"
            else:
                cursor.execute(query)
                connection.commit()
                return "Query executed successfully"
        except Error as e:
            return "Error occurred: " + str(e)


    def load_shot_data_reader(self,connection,shots_file):

        all_shots = import_shot_game_data(shots_file) 
        #all_shots = get_shots_file_path()
        
        all_shifts = import_shift_data(TEST_GAME_ID, TEST_GAME_SEASON)

        #if all_shots and all_shifts:
        if all_shifts:
            for x in all_shots:

                players_for_id = []
                players_against_id = []
                
                players_for, players_against = get_players_on_ice_for_shot(x, all_shifts)
                #print(players_for['id'])
                #print(players_against['id'])
                for player in players_for:
                    #print(player['id'])
                    if player['id'] not in players_for_id:
                        players_for_id.append(player['id'])

                for player in players_against:
                    if player['id'] not in players_against_id:
                        players_against_id.append(player['id'])
                        


                #print(players_for_id)
                players_for_id.sort()
                players_against_id.sort()

                print(players_for_id)
                print(players_against_id)



                while len(players_for_id) < 6:
                    players_for_id.append(-1)
                while len(players_against_id) < 6:
                    players_against_id.append(-1)
                meta = MetaData()
                meta.reflect(bind=engine)
                poiu_table = meta.tables['poiu']
                try:
                    insert_table = insert(poiu_table).values(playerid1=players_for_id[0],playerid2=players_for_id[1],playerid3=players_for_id[2],playerid4=players_for_id[3],playerid5=players_for_id[4],playerid6=players_for_id[5])
                    insert_table = insert_table.on_conflict_do_nothing(
                        #index_elements=['playerid1','playerid2','playerid3','playerid4','playerid5','playerid6']
                        #index_elements=['poiu_unq']
                        )
                    conn.execute(insert_table)
                except Exception as e:
                 print("Error in Querying Total Shots")
                 print(e)
                #connection.commit()
        
        
        conn.commit()
        #conn.close()
       ######################################################################


meta = MetaData()
meta.reflect(bind=engine)
poiu_table = meta.tables['poiu']
print(poiu_table.columns)

In [ ]:
if __name__ == "__main__":
    
    #print('\033[32m' + "Shot and Shift Data Upload: " + '\033[m')
    db = shot_db_sql()


    try:
        conn = db.create_connection()
    except:
        print("Database Connection Error")

    shots_file = get_shots_file_path()

    try:
        print('\033[32m' + "Total shots loaded: " + '\033[m' + str(db.load_shot_data_reader(conn,shots_file)))
    except Exception as e:
         print("Error in Querying Total Shots")
         print(e)


    conn.close()

In [ ]:
#connection = psycopg2.connect(database= "POIU_Analytics", user="tyler", host="127.0.0.1", port="5050")